# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
#Read in weatherpy dataframe
wetdf = pd.read_csv("../output_data/weather.csv")
wetdf.head(10)


,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Windspeed,Date
0,bluff,NZ,-46.6000,168.3333,45.81,83,75,15.82,1627315253
1,bengkulu,ID,-3.8004,102.2655,75.16,64,76,4.29,1627315253
2,pevek,RU,69.7008,170.3133,35.38,81,89,12.15,1627315254
3,carnarvon,AU,-24.8667,113.6333,69.87,75,85,14.97,1627315010
4,makakilo city,US,21.3469,-158.0858,76.75,75,73,4.61,1627315009
5,tabas,IR,33.5959,56.9244,99.52,1,14,24.85,1627315254
6,tuktoyaktuk,CA,69.4541,-133.0374,42.69,98,80,6.02,1627315255
7,butaritari,KI,3.0707,172.7902,81.88,100,73,6.91,1627315255
8,muhos,FI,64.8080,25.9931,73.18,9,52,6.13,1627315256
9,busselton,AU,-33.6500,115.3333,57.09,48,61,28.72,1627315256


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# grab maps
gmaps.configure(api_key=g_key)
# store lats/longs and humidity
Locations = wetdf[["Lat", "Lng"]]
hum = wetdf["Humidity"].astype(float)

In [17]:
# Create heatmap
ax = gmaps.figure(center = (46.0, -5.0), zoom_level=2)
max_intensity = np.max(hum)
# Create heat layer
heat = gmaps.heatmap_layer(Locations, weights = hum, max_intensity=100, point_radius=3, dissipating=False)
# Add heat layer and display map
ax.add_layer(heat)
ax

Figure(layout=FigureLayout(height='420px'))

In [18]:
max_intensity

100.0

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# Narrow down dataframe to cities w/ windspeed under 10 mph, no clouds, temp between 60-70 degrees F --> display
narrowdf = wetdf.loc[(wetdf["Windspeed"] <= 10) & (wetdf["Cloudiness"] == 0) & \
                     (wetdf["Max Temp"] >= 60) & (wetdf["Max Temp"] <= 70)].dropna()
narrowdf.head(10)

,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Windspeed,Date
47,port elizabeth,ZA,-33.9180,25.5701,69.10,0,45,1.14,1627315182
55,belyy yar,RU,53.6039,91.3903,69.69,0,88,0.00,1627315274
60,thompson,CA,55.7435,-97.8558,66.13,0,27,5.84,1627315276
94,nokaneng,BW,-19.6667,22.2667,63.82,0,26,6.62,1627315288
199,pisco,PE,-13.7000,-76.2167,67.01,0,81,6.85,1627315330
219,dongsheng,CN,39.8161,109.9776,69.44,0,57,5.97,1627315338
526,la rioja,AR,-29.4111,-66.8507,60.46,0,35,7.25,1627315452


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
# Create hotel_df
hotel_df = narrowdf.loc[:,["City","Country", "Lat", "Lng"]]
# add a "Hotel Name" column
hotel_df["Hotel Name"] = ""
# Display the dataframe
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
47,port elizabeth,ZA,-33.9180,25.5701,
55,belyy yar,RU,53.6039,91.3903,
60,thompson,CA,55.7435,-97.8558,
94,nokaneng,BW,-19.6667,22.2667,
199,pisco,PE,-13.7000,-76.2167,
219,dongsheng,CN,39.8161,109.9776,
526,la rioja,AR,-29.4111,-66.8507,


In [21]:
# set base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#add parameters
params = {"type" : "hotel", "keyword" : "hotel", "radius" : 5000, "key" : g_key}

In [22]:
#loop through hotel dataframe
for index, row in hotel_df.iterrows():
    # get lat long, add parameters
    lng = row["Lng"]
    lat = row["Lat"]
    name = row["City"]
    params["location"] = f"{lat},{lng}"
    # Make request
    print(f"Beginning retrieval for Index {index}: {name}.")
    response = requests.get(base_url, params=params).json()
    result = response['results']
    # save results to hotel dataframe
    try:
        print(f"The closest hotel in {name} is {result[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = result[0]['name']
    #Except if ther is no hotel, skip
    except (KeyError, IndexError):
        print("No result, skipping.")
    #Wait a sec to avoid ssl
    time.sleep(1)
print('''
-----------------------------
Data Retrieval Complete
-----------------------------''')

Beginning retrieval for Index 47: port elizabeth.
The closest hotel in port elizabeth is Island Way Villa.
Beginning retrieval for Index 55: belyy yar.
No result, skipping.
Beginning retrieval for Index 60: thompson.
The closest hotel in thompson is Best Western Thompson Hotel & Suites.
Beginning retrieval for Index 94: nokaneng.
No result, skipping.
Beginning retrieval for Index 199: pisco.
The closest hotel in pisco is PiscoMar Peru.
Beginning retrieval for Index 219: dongsheng.
The closest hotel in dongsheng is Holiday Inn Express Ordos Dongsheng.
Beginning retrieval for Index 526: la rioja.
The closest hotel in la rioja is Reina Mora Hotel.

-----------------------------
Data Retrieval Complete
-----------------------------


In [26]:
# display changes
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
47,port elizabeth,ZA,-33.9180,25.5701,Island Way Villa
55,belyy yar,RU,53.6039,91.3903,
60,thompson,CA,55.7435,-97.8558,Best Western Thompson Hotel & Suites
94,nokaneng,BW,-19.6667,22.2667,
199,pisco,PE,-13.7000,-76.2167,PiscoMar Peru
219,dongsheng,CN,39.8161,109.9776,Holiday Inn Express Ordos Dongsheng
526,la rioja,AR,-29.4111,-66.8507,Reina Mora Hotel


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
ax.add_layer(markers)

# Display figure
ax

Figure(layout=FigureLayout(height='420px'))